In [1]:
import matplotlib.pyplot as plt
import numpy as np

# Read Input Image

In [2]:
file_image1 = "image1"
image1 = plt.imread(file_image1 + ".jpg")

# Face Detection

## Detect

In [3]:
from source.face_detection import FaceDetector

fd = FaceDetector(minimum_confidence=0.95, output_size=250)

images_faces = fd.compute([image1]).get_images_faces()

## Results

In [4]:
fd.get_images_faces_scores()

[array([0.9999987, 0.9999993, 0.9990159], dtype=float32)]

In [5]:
fd.get_bounding_boxes()

[array([[1479,  829, 1859, 1315],
        [1983, 1117, 2294, 1497],
        [2512, 1116, 2791, 1459]], dtype=int16)]

In [6]:
fd.get_facial_keypoints()

[array([[[1628, 1022],
         [1792, 1061],
         [1714, 1149],
         [1601, 1190],
         [1741, 1225]],
 
        [[2084, 1263],
         [2221, 1268],
         [2162, 1337],
         [2096, 1409],
         [2206, 1416]],
 
        [[2604, 1259],
         [2738, 1268],
         [2676, 1325],
         [2601, 1378],
         [2716, 1387]]], dtype=int16)]

In [7]:
for i, faces in enumerate(images_faces):
    for j, face in enumerate(faces):
        plt.imsave(
            f"{file_image1}_image_index{i}_face_{j}.jpg",
            face
        )


# Face Alignment

## Align

In [8]:
from source.face_alignment import FaceAligner

fa = FaceAligner()
aligned_images_faces = fa.align_images_faces(images_faces, fd.get_eyes_coordinates())


## Results

In [9]:
for i, image_faces in enumerate(aligned_images_faces):
    for j, face in enumerate(image_faces):
        plt.imsave(
            f"{file_image1}_rotated_image_index{i}_face_{j}.jpg",
            face
        )


# Face Normalization

## Normalize

In [10]:
len(aligned_images_faces), len(aligned_images_faces[0]), aligned_images_faces[0][0].shape

(1, 3, (250, 250, 3))

In [11]:
from source.face_normalization import FaceNormalizer

fn = FaceNormalizer()
normalized_aligned_images_faces = fn.normalize_images_faces(aligned_images_faces)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Results

In [12]:
for i, image_faces in enumerate(normalized_aligned_images_faces):
    for j, face in enumerate(image_faces):
        plt.imsave(
            f"{file_image1}_normalized_rotated_image_index{i}_face_{j}.jpg",
            face
        )


# Facial Emotion Recognition
## Face Emotion Representation

In [13]:
from source.face_emotion_recognition import FaceEmotionRecognizer

model_name = 'enet_b0_8_best_afew'
# model_name='enet_b0_8_best_vgaf'
# model_name='enet_b0_8_va_mtl'
# model_name='enet_b2_8'
fer = FaceEmotionRecognizer(model_name, device="cuda:0")
images_faces_representations = [fer.extract_representations_from_image_faces(normalized_aligned_image_faces) for normalized_aligned_image_faces in normalized_aligned_images_faces
                                ]

/home/ahur4/.hsemotions/enet_b0_8_best_afew.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [14]:
images_faces_representations

[tensor([[-0.0864, -0.0695,  0.3409,  ..., -0.0926,  0.0393,  0.0659],
         [ 0.1862, -0.1005,  0.3144,  ...,  0.1115, -0.1877,  0.0243],
         [ 0.0773,  0.0140,  0.5271,  ..., -0.0428,  0.1012, -0.1217]],
        device='cuda:0', grad_fn=<ReshapeAliasBackward0>)]

In [15]:
# 1 image, 3 face in this image, each face representation is vector of (1, 1280) shape
len(images_faces_representations), images_faces_representations[0].shape

(1, torch.Size([3, 1280]))

## Face Emotion Recognition

In [16]:
images_faces_predictions_labels_scores = [fer.predict_emotions_from_image_faces_representations(image_faces_representations) for image_faces_representations in images_faces_representations]

## Results

In [17]:
images_faces_predictions_labels_scores

[(['Happiness', 'Anger', 'Fear'],
  tensor([[-2.3273,  0.8328, -0.6988,  0.6483,  3.2102,  1.4981, -0.0774, -0.3872],
          [ 5.6108, -0.5087,  0.4402,  1.7418, -2.9160,  1.7116,  2.6144, -1.6169],
          [-2.3844, -2.7352,  0.3890,  2.6326,  0.9862, -0.6138,  1.8679, -0.2901]],
         device='cuda:0', grad_fn=<AddBackward0>))]

---

# Representation Extraction

In [18]:
from source.representation_extraction import EmotionRepresentationExtractor
from source.face_detection import FaceDetector
from source.face_alignment import FaceAligner
from source.face_normalization import FaceNormalizer
from source.face_emotion_recognition import FaceEmotionRecognizer
import matplotlib.pyplot as plt


## Read Images

In [19]:
file_image1 = "image1"
image1 = plt.imread(file_image1 + ".jpg")

## Extract

In [20]:
fd = FaceDetector(minimum_confidence=0.95, output_size=250)
fa = FaceAligner()
fn = FaceNormalizer()
model_name = 'enet_b0_8_best_afew'
# model_name='enet_b0_8_best_vgaf'
# model_name='enet_b0_8_va_mtl'
# model_name='enet_b2_8'
fer = FaceEmotionRecognizer(model_name, device="cuda:0")

fre = EmotionRepresentationExtractor().set_face_detection_model(fd, ).set_face_alignment_model(fa, ).set_face_normalizer_model(fn).set_face_emotion_recognition_model(fer)

/home/ahur4/.hsemotions/enet_b0_8_best_afew.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [21]:
images_faces_representations = fre.extract_representation([image1])
images_faces_representations # Next Step: Mean

[tensor([[-0.0864, -0.0695,  0.3409,  ..., -0.0926,  0.0393,  0.0659],
         [ 0.1862, -0.1005,  0.3144,  ...,  0.1115, -0.1877,  0.0243],
         [ 0.0773,  0.0140,  0.5271,  ..., -0.0428,  0.1012, -0.1217]],
        device='cuda:0', grad_fn=<ReshapeAliasBackward0>)]